In [1]:
import pandas as pd
import numpy as np
import datetime
import sunpy.map
import sunpy.data.sample
from sunpy.net import Fido, jsoc, attrs as a
import astropy.units as u

In [9]:
df = pd.read_excel('eruptedFilamentsList.xlsx', sheet_name='Sheet1')  
SOLstr = df.iloc[1::2, 0].tolist()
# 提取时间部分并转换为 datetime 类型
time_ls = [pd.to_datetime(s[3:22], format='%Y-%m-%dT%H:%M:%S') for s in SOLstr]


In [4]:
def union(lst):
    result = lst[0]
    for i in range(len(lst)):
        result |= lst[i]
    return result

In [ ]:
import time
client = jsoc.JSOCClient()
# 设置时间范围
time_all = []
for i in range(len(time_ls)):
    time_range = a.Time(time_ls[i] - datetime.timedelta(seconds=31), time_ls[i] - datetime.timedelta(seconds=30))
    time_all.append(time_range)

# 设置搜索条件
for i in range(len(time_all)):
    result = client.search(time_all[i],
                        a.jsoc.Series('hmi.M_45s'),
                        a.jsoc.Notify('211503020@smail.nju.edu.cn'),
                        a.jsoc.Segment('magnetogram'))
    while True:
        try:
            requests0 = client.request_data(result)
            res = client.get_request(requests0, path='D:\\labeledFilaments\\111')
            break
        except:
            time.sleep(5)
        


In [10]:
# 原时间序列中有无时间过近的情况
save_1 = []
for i in range(len(time_ls)-1):
    delta = time_ls[i+1] - time_ls[i]
    if (delta < datetime.timedelta(seconds = 120))&(delta != datetime.timedelta(0)):
        print(delta)
        save_1.append(time_ls[i])
        save_1.append(time_ls[i+1])
        i += 1

time_ls_set = set(time_ls)
time1_ls = list(time_ls_set)

print(len(time_ls_set))
time1_ls = sorted(time1_ls)

538


In [38]:
# 查找已下载数据中过近的
import os

folder = 'D:\\labeledFilaments\\111'
file_ls = os.listdir(folder)
file_ls = sorted(file_ls)
total = len(file_ls)
file_time_ls = [pd.to_datetime(s[10:25], format='%Y%m%d_%H%M%S') for s in file_ls]
file_time_ls = sorted(file_time_ls)
save_2 = []
save_index = []
for i in range(len(file_time_ls)-1):
    delta = file_time_ls[i+1] - file_time_ls[i]
    if (delta < datetime.timedelta(seconds = 60)):

        save_2.append(file_time_ls[i])
        save_2.append(file_time_ls[i+1])
        save_index.append(i+1)
print(len(save_2))
print(len(save_index))
# rm_names = [file_ls[i] for i in save_index]
# for name in rm_names:
#     os.remove(os.path.join(folder, name))

0
0


In [56]:
print(time1_ls[472])
print(file_ls[431])  # 缺少timels 431(2014-07-09 17:06:03) 472(2015-02-04 18:00:03)

2015-02-04 18:00:03
hmi.m_45s.20140718_000000_TAI.2.magnetogram.fits


In [47]:
# for i in range(len(file_time_ls)):
#     if (file_time_ls[i] - time1_ls[i]) > datetime.timedelta(minutes= 10):
#         print(i)
#         print(file_time_ls[i])
#         print(time1_ls[i])
#         print(file_ls[i])
#         print('-----------------')

for j in range(471, len(file_time_ls)-2):
    if ((file_time_ls[j] - time1_ls[j+2]) > datetime.timedelta(minutes= 10))|((file_time_ls[j] - time1_ls[j+1]) < datetime.timedelta(minutes= -10)):
        print(j)
        print(file_time_ls[j])
        print(time1_ls[j+2])
        print(file_ls[j])
        print('-----------------')

In [8]:
# 查漏补缺
import time
client = jsoc.JSOCClient()
result1 = client.search(a.Time('2011-04-02T08:59:30', '2011-04-02T09:01:00'),
                        a.jsoc.Series('hmi.M_45s'),
                        a.jsoc.Notify('211503020@smail.nju.edu.cn'),
                        a.jsoc.Segment('magnetogram'))
    
requests1 = client.request_data(result1)
print(result1)
res1 = client.get_request(requests1, path='D:\\labeledFilaments\\112')

         T_REC          TELESCOP INSTRUME WAVELNTH CAR_ROT
----------------------- -------- -------- -------- -------
2011.04.02_09:00:00_TAI  SDO/HMI  MISSING   6173.0      --
2011.04.02_09:00:45_TAI  SDO/HMI  MISSING   6173.0      --
2011.04.02_09:01:30_TAI  SDO/HMI  MISSING   6173.0      --
1 URLs found for download. Full request totalling 0MB


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

In [59]:
# 将ttime1_ls写入文件
with open('time1_ls.txt', 'w') as f:
    for i in range(len(time1_ls)):
        if i != 431 | 472:
            f.write(str(i) + '  ' + str(time1_ls[i]) + '\n')
print(len(time1_ls) - 2)
print(len(file_time_ls))

536
536


In [86]:
# 获取查询结果的时间戳数据
t_rec = result['T_REC']

# 将时间戳数据转换为一个列表
t_rec_list = t_rec.tolist()

t_rec_list = pd.to_datetime(t_rec_list, format='%Y.%m.%d_%H:%M:%S_TAI')

min_index = np.argmin(abs(t_rec_list - target_time))

# 获取与指定时间最接近的结果
result_row = result[min_index].as_table()

result_row.query_args

[{'start_time': <Time object: scale='utc' format='datetime64' value=2010-05-26T06:18:27.000000000>,
  'end_time': <Time object: scale='utc' format='datetime64' value=2010-05-26T06:18:28.000000000>,
  'series': 'hmi.M_45s',
  'notify': '211503020@smail.nju.edu.cn',
  'segment': ['magnetogram']}]

In [87]:

# 请求数据
requests = client.request_data(result_row)
res = client.get_request(requests)
res.wait(progress=True)


1 URLs found for download. Full request totalling 14MB


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

hmi.m_45s.20100526_061845_TAI.2.magnetogram.fits:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

AttributeError: 'Results' object has no attribute 'wait'